In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
#load modules
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from astropy import units as u
from astropy.io import fits
from  matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import scipy.ndimage

# draw the dark hole regions
from skimage.draw import draw

# import personal code
import sys
sys.path.insert(0, '/home/jhen/XWCL/code/MagAOX/functions')
import datafiles as dfx

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from datetime import datetime
date_ran = datetime.now()
dr_str = date_ran.strftime("%d/%m/%Y %H:%M:%S")
print('Last date this notebook has been ran: {0}'.format(dr_str))

Last date this notebook has been ran: 23/04/2020 11:51:15


In [2]:
##### investigate .datx files
coat_labels =  ('0', '1', '2', '3', '4', '5_sn1', '5_sn2', '5_sn3')
mat_sizes = []

# find the smallest size to calibrate all the optics to this size
for cl in coat_labels:
    fileloc = 'oaps/oap_coated/uncentered_data/oap{0}_mx_ca.datx'.format(cl)
    surf, mask, sp = dfx.open_datx(datx_file_loc=fileloc)
    surf, mask = dfx.mat_tight(surf,mask)
    latres = sp['value'][sp['label'].index('latres')]
    diam = (latres * mask.shape[0]*u.pix).to(u.mm)
    print('n{0:5} | diameter: {1:.3f} | shape: {2}'.format(cl, diam, mask.shape))
    mat_sizes.append(np.shape(mask))
min_side = np.amin(mat_sizes) # pray this is reasonable
print('min side: {0}'.format(min_side))

n0     | diameter: 40.328 mm | shape: (773, 773)
n1     | diameter: 40.432 mm | shape: (775, 775)
n2     | diameter: 40.432 mm | shape: (775, 775)
n3     | diameter: 40.432 mm | shape: (775, 775)
n4     | diameter: 40.432 mm | shape: (775, 775)
n5_sn1 | diameter: 40.432 mm | shape: (775, 775)
n5_sn2 | diameter: 40.432 mm | shape: (775, 775)
n5_sn3 | diameter: 40.432 mm | shape: (775, 775)
min side: 773


In [3]:
# resize all the data to one shape, interpolate bad data present

fits_folder = 'oaps/oap_coated/'
undersize = 4 # undersize to whole matrix

for cl in coat_labels:
    # call in the file
    fileloc = 'oaps/oap_coated/uncentered_data/oap{0}_mx_ca.datx'.format(cl)
    surf, mask, sp = dfx.open_datx(datx_file_loc=fileloc, diam_ca100=50*u.mm)

    # tighten up the matrix by removing empty rows and columns
    surf, mask = dfx.mat_tight(surf, mask)
    
    # undersize the matrix equally
    side_cut = mask.shape[0] - min_side + undersize
    if (mask.shape[0] - side_cut) % 2 != 0: # odd new matrix size
        side_cut += 1 # force it even
    surf, mask = dfx.mat_reduce(data=surf, mask=mask, side_reduce=side_cut)
    
    latres = sp['value'][sp['label'].index('latres')]
    diam = (latres * mask.shape[0]*u.pix).to(u.mm)
    clear_ap = diam / sp['value'][sp['label'].index('diam_100')] * 100
    print('n{0:5} | diameter: {1:.3f} | clear ap: {2:.2f}% | shape: {3}'.format(cl, diam, clear_ap, mask.shape))
    
    # check that there is nothing missing in the data
    if cl == '0': # it's cheating but it's known that the 1st OAP has no bad data inside
        ap_clear = mask
        ap_coords = np.where(mask==1)
    # set up fits header cards
    sp['label'].append('INTERP') # add fits header comment
    sp['comment'].append('True if interpolated bad surf pixels')
    # investigate inside the good aperture
    if (np.where(mask[ap_coords]==0))[0].size >0: # we got bad data
        n_badpix = np.shape(np.where(mask[ap_coords]==0))[1]
        print('OAP{0} has bad data: {1} bad pixels present. Interpolating to fix.'.format(cl, n_badpix))
        sp['value'].append(True)
        new_surf = dfx.fill_surface(surf, mask, ap_clear, ap_coords)
    else:
        print('OAP{0} has all good data'.format(cl))
        sp['value'].append(False)
        new_surf = surf*ap_clear
        
    # write to file
    new_filename = 'oap{0}_centered_{1}CA'.format(cl, int(clear_ap))
    dfx.write_fits(surface=surf, mask=ap_clear, surf_parms=sp, filename=fits_folder+new_filename)
    print('OAP{0} file has been written: {1}\n'.format(cl, new_filename))

n0     | diameter: 40.067 mm | clear ap: 80.13% | shape: (768, 768)
OAP0 has all good data
OAP0 file has been written: oap0_centered_80CA

n1     | diameter: 40.067 mm | clear ap: 80.13% | shape: (768, 768)
OAP1 has all good data
OAP1 file has been written: oap1_centered_80CA

n2     | diameter: 40.067 mm | clear ap: 80.13% | shape: (768, 768)
OAP2 has all good data
OAP2 file has been written: oap2_centered_80CA

n3     | diameter: 40.067 mm | clear ap: 80.13% | shape: (768, 768)
OAP3 has all good data
OAP3 file has been written: oap3_centered_80CA

n4     | diameter: 40.067 mm | clear ap: 80.13% | shape: (768, 768)
OAP4 has all good data
OAP4 file has been written: oap4_centered_80CA

n5_sn1 | diameter: 40.067 mm | clear ap: 80.13% | shape: (768, 768)
OAP5_sn1 has bad data: 27 bad pixels present. Interpolating to fix.
OAP5_sn1 file has been written: oap5_sn1_centered_80CA

n5_sn2 | diameter: 40.067 mm | clear ap: 80.13% | shape: (768, 768)
OAP5_sn2 has all good data
OAP5_sn2 file has 